In [1]:
using TuringModels

Turing.setadbackend(:reverse_diff)
#nbTuring.turnprogress(false);

d = CSV.read(rel_path("..", "data", "UCBadmit.csv"), delim=';');
size(d) # Should be 12x5

(12, 5)

Turing model

In [2]:
@model m11_5(admit, applications) = begin
    N=length(applications)
    θ ~ Truncated(Exponential(1), 0, Inf)
    α ~ Normal(0,2)

    for i ∈ 1:N
        prob = logistic(α)
        # alpha and beta for the BetaBinomial must be provided.
        # The two parameterizations are related by
        # alpha = prob * theta, and beta = (1-prob) * theta.
        # See https://github.com/rmcelreath/rethinking/blob/master/man/dbetabinom.Rd
        alpha = prob * θ
        beta = (1 - prob) * θ

        admit[i] ~ BetaBinomial(applications[i], alpha, beta)
    end
end

m11_5 (generic function with 3 methods)

Sample

In [3]:
posterior = sample(m11_5(d[:admit],d[:applications]), Turing.NUTS(4000, 1000, 0.9));

┌ Info: [Turing] looking for good initial eps...
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/support/hmc_core.jl:240
[NUTS{Turing.Core.FluxTrackerAD,Union{}}] found initial ϵ: 0.8
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/support/hmc_core.jl:235
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:168
┌ Warning: grad = [NaN, NaN]
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:169
┌ Info:  Adapted ϵ = 0.15907376932603656, std = [1.0, 1.0]; 1000 iterations is used for adaption.
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/adapt/adapt.jl:91
[NUTS] Finished with
  Running time        = 22.059370601000015;
  #lf / sample        = 0.0;
  #evals / sample     = 17.42025;
  pre-cond. metric    = [1.0, 1.0].


Draw summary

In [4]:
describe(posterior)

Iterations = 1:4000
Thinning interval = 1
Chains = 1
Samples per chain = 4000

Empirical Posterior Estimates:
              Mean           SD        Naive SE       MCSE         ESS   
       α  -0.3727658956 0.316627690 0.00500632336 0.0065444670 2340.71866
  lf_num   0.0000000000 0.000000000 0.00000000000 0.0000000000        NaN
 elapsed   0.0055148427 0.011580373 0.00018310177 0.0002359631 2408.55697
 epsilon   0.1873475827 0.116654900 0.00184447591 0.0079570227  214.93383
      lp -71.1203546921 1.078809709 0.01705747921 0.0305501087 1246.99345
eval_num  17.4202500000 7.494676943 0.11850124734 0.3393580832  487.74120
       θ   2.7718770502 0.985180746 0.01557707533 0.0252877601 1517.78797
  lf_eps   0.1873475827 0.116654900 0.00184447591 0.0079570227  214.93383

Quantiles:
               2.5%           25.0%         50.0%          75.0%         97.5%    
       α  -0.98699232746  -0.5838617475  -0.376888910  -0.1659742742   0.267363481
  lf_num   0.00000000000   0.0000000000   0.00

Result rethinking

In [5]:
m115rethinking = "
         mean   sd  5.5% 94.5% n_eff Rhat
theta  2.74 0.96  1.43  4.37  3583    1
a       -0.37 0.31 -0.87  0.12  3210    1
";

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*